In [143]:
# Connecting to the db
import lib.handle_db as dbh

# Parsing html 
from bs4 import BeautifulSoup

# http requests 
import requests

exclude_metadata = {'nature catalysis':['viewport', 'msapplication-TileColor', 'msapplication-config',  'theme-color', 
                    'application-name', 'robots', 'access', 'WT.cg_s', 'WT.z_bandiera_abtest', 'WT.page_categorisation',
                    'WT.template', 'WT.z_cg_type', 'WT.cg_n', 'dc.rights', 'prism.issn']}

section_labels = {'nature catalysis':{'aria-labelledby':'data-availability'}}

def get_metadata(uri, journal):
    result=[]
    try:
        req_head = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'}
        response = requests.get(uri, headers = req_head)
        soup = BeautifulSoup(response.text,'html.parser')       
        metadata = soup.find_all('meta')
        ignore_these = []
        if journal in exclude:
            ignore_these = exclude_metadata[journal] 
        else:
            print('new journal')
        for md_item in metadata:
            if md_item.has_attr("name") and not md_item["name"] in ignore_these :
                result.append(md_item)
    except Exception as e:
        print(e)
    return result


In [144]:
url = "https://www.nature.com/articles/s41929-019-0334-3"
# get metadata

def get_data_from_metadata(url, journal = 'nature catalysis'):
    ret_data = ""
    res = get_metadata(url, 'nature catalysis')
    # check if metadata references supporting data or supplementary data
    for md_item in res:
        if 'data' in str(md_item["name"]).lower():
            #print(md_item["name"], md_item["content"])
            ret_data = md_item["content"]
    # get author(s) data from metadata
    #for md_item in res:
    #    if 'author' in str(md_item["name"]).lower():
    #        print(md_item["name"], md_item["content"])    
    return ret_data
    

Not found in metadata


In [148]:
def get_data_from_section(uri, journal = 'nature catalysis'):
    data_link = ""
    for section in sections:
        inspect_this = section_labels[journal]
        for sect_attr in inspect_this:
            if section.has_attr(sect_attr) and section[sect_attr] == 'data-availability':
                #print(section["aria-labelledby"])
                pars = section.find_all('p')
                for par in pars:
                    #print(par)
                    references = par.find_all('a')
                    for reference in references:
                        for content in reference.contents:
                            #print(content)
                            data_link = content
    return data_link

In [151]:

res = get_data_from_metadata(url, journal = 'nature catalysis')
if res != "":
    print(res)
else:
    print("Not found in metadata")
    
    
res = get_data_from_section(url, 'nature catalysis')

if res != "":
    print("Found as section: ", res)
else:
    print("Not found in metadata")


Not found in metadata
Found as section:  https://doi.org/10.17035/d.2019.0079744472


In [130]:
res

'https://doi.org/10.17035/d.2019.0079744472'

In [ ]:
req_head = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'}
response = requests.get(url, headers = req_head)
soup = BeautifulSoup(response.text,'html.parser')  
sections = soup.find_all('section')

